In [1]:
from sklearn.ensemble import GradientBoostingClassifier
import pickle
from sklearn.metrics import ndcg_score
import pandas as pd
import gc

In [2]:
# training set
x = pd.read_csv("data/dummy/train.csv")

# val set 20%
x_val = x.sample(n=int(0.1*len(x)))
x = x.drop(x_val.index)

## TODO: date_time
x.drop(['click_bool', 'booking_bool', 'date_time'], axis=1, inplace=True)
x_val.drop(['click_bool', 'booking_bool', 'date_time'], axis=1, inplace=True)

# sort both on search id
x.sort_values('srch_id', ignore_index=True)
x_val.sort_values('srch_id', ignore_index=True)

,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,...,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,orig_destination_distance,random_bool,agg_comp_rate,agg_comp_perc,agg_comp_inv,target
0,1,12,187,219,53341,4,4.0,1,2.56,0.1238,...,4,0,1,1,-1.00,1,0.0,6.0,0.0,0
1,1,12,187,219,56880,4,4.0,1,2.83,0.1028,...,4,0,1,1,-1.00,1,0.0,0.0,0.0,0
2,4,5,219,219,110813,2,4.0,0,2.40,0.0000,...,1,0,1,1,233.87,1,0.0,0.0,0.0,0
3,8,5,219,219,54403,2,3.5,1,0.00,0.0000,...,1,0,1,0,8.77,0,0.0,0.0,0.0,0
4,8,5,219,219,39137,2,3.5,1,1.61,0.0813,...,1,0,1,0,4.85,0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495829,332782,19,158,158,18628,4,4.0,1,0.69,0.0575,...,1,0,1,1,-1.00,0,0.0,0.0,0.0,0
495830,332782,19,158,158,26348,4,0.0,1,0.00,0.0000,...,1,0,1,1,-1.00,0,0.0,0.0,0.0,0
495831,332782,19,158,158,74974,2,2.0,0,2.77,0.0446,...,1,0,1,1,-1.00,0,0.0,0.0,0.0,0
495832,332782,19,158,158,9869,0,0.0,0,2.64,0.0072,...,1,0,1,1,-1.00,0,0.0,0.0,0.0,0


In [3]:
y_val = x_val.target.squeeze()
y = x.target.squeeze()
x.drop(['target'], axis=1, inplace=True)
x_val.drop(['target'], axis=1, inplace=True)

In [4]:
groups = x.srch_id.value_counts().sort_index()
groups_val = x_val.srch_id.value_counts().sort_index()

In [5]:
import lightgbm

# default lightgbm model with sklearn api
gbm = lightgbm.LGBMRanker(metric="ndcg", objective="lambdarank") 

# fitting model 
gbm.fit(
    x,
    y,
    eval_set = [(x_val, y_val)],
    group = groups,
    eval_group = [groups_val],
    verbose=1,
    early_stopping_rounds=100
)

[1]	valid_0's ndcg@1: 0.93995	valid_0's ndcg@2: 0.958314	valid_0's ndcg@3: 0.966594	valid_0's ndcg@4: 0.970239	valid_0's ndcg@5: 0.97174
Training until validation scores don't improve for 100 rounds
[2]	valid_0's ndcg@1: 0.941695	valid_0's ndcg@2: 0.960221	valid_0's ndcg@3: 0.968095	valid_0's ndcg@4: 0.971435	valid_0's ndcg@5: 0.972856
[3]	valid_0's ndcg@1: 0.942854	valid_0's ndcg@2: 0.961035	valid_0's ndcg@3: 0.968706	valid_0's ndcg@4: 0.972005	valid_0's ndcg@5: 0.973384
[4]	valid_0's ndcg@1: 0.94354	valid_0's ndcg@2: 0.961332	valid_0's ndcg@3: 0.969094	valid_0's ndcg@4: 0.972317	valid_0's ndcg@5: 0.973654
[5]	valid_0's ndcg@1: 0.943856	valid_0's ndcg@2: 0.96162	valid_0's ndcg@3: 0.969349	valid_0's ndcg@4: 0.972481	valid_0's ndcg@5: 0.973823
[6]	valid_0's ndcg@1: 0.943877	valid_0's ndcg@2: 0.961618	valid_0's ndcg@3: 0.969342	valid_0's ndcg@4: 0.972548	valid_0's ndcg@5: 0.973846
[7]	valid_0's ndcg@1: 0.944083	valid_0's ndcg@2: 0.961736	valid_0's ndcg@3: 0.969413	valid_0's ndcg@4: 0.972

[61]	valid_0's ndcg@1: 0.944667	valid_0's ndcg@2: 0.962375	valid_0's ndcg@3: 0.969839	valid_0's ndcg@4: 0.972984	valid_0's ndcg@5: 0.974271
[62]	valid_0's ndcg@1: 0.944787	valid_0's ndcg@2: 0.96244	valid_0's ndcg@3: 0.969866	valid_0's ndcg@4: 0.97303	valid_0's ndcg@5: 0.974316
[63]	valid_0's ndcg@1: 0.944804	valid_0's ndcg@2: 0.962441	valid_0's ndcg@3: 0.969864	valid_0's ndcg@4: 0.973035	valid_0's ndcg@5: 0.974322
[64]	valid_0's ndcg@1: 0.944838	valid_0's ndcg@2: 0.962459	valid_0's ndcg@3: 0.969889	valid_0's ndcg@4: 0.973043	valid_0's ndcg@5: 0.97434
[65]	valid_0's ndcg@1: 0.944815	valid_0's ndcg@2: 0.962425	valid_0's ndcg@3: 0.969862	valid_0's ndcg@4: 0.973012	valid_0's ndcg@5: 0.974321
[66]	valid_0's ndcg@1: 0.944758	valid_0's ndcg@2: 0.962436	valid_0's ndcg@3: 0.969846	valid_0's ndcg@4: 0.973003	valid_0's ndcg@5: 0.974306
[67]	valid_0's ndcg@1: 0.944786	valid_0's ndcg@2: 0.962452	valid_0's ndcg@3: 0.96988	valid_0's ndcg@4: 0.973022	valid_0's ndcg@5: 0.974318
[68]	valid_0's ndcg@1: 0

LGBMRanker(metric='ndcg', objective='lambdarank')

In [6]:
gc.collect()
with open('model/model', 'wb') as f:
    pickle.dump(gbm, f)

In [7]:
with open('model/model', 'rb') as f:
    model = pickle.load(f)

In [8]:
test = pd.read_csv("data/dummy/test.csv", index_col=False)
test.drop('date_time', axis=1, inplace=True)

In [9]:
predictions = model.predict(test)

In [10]:
len(predictions)

4959183

In [11]:
submission = test[["srch_id", "prop_id"]]
submission['pred'] = predictions

<ipython-input-11-f49d9f7448f2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['pred'] = predictions


In [12]:
del test
gc.collect()

84

In [13]:
submission = submission.sort_values(['srch_id', 'pred'], ascending=[True, False])

In [14]:
submission.head(n=20)

,srch_id,prop_id,pred
23,1,99484,1.475379
9,1,54937,1.275869
12,1,61934,1.231074
4,1,24194,0.881236
5,1,28181,0.877300
6,1,34263,0.642461
8,1,50162,0.589785
18,1,82231,0.515430
20,1,90385,0.483768
13,1,63894,0.466325


In [15]:
submission[['srch_id', 'prop_id']].to_csv('submission.csv', index=False)